# Logic:
* Normalize per param & per year
* Assign weights to proxies
* Calculate scores

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
DATE = datetime.today().strftime("%Y-%m-%d")
NEGATIVES = ["SH.DYN.MORT"]
PROXIES_WEIGHT = {
    "MS.MIL.TOTL.TF.ZS": 1.25,
    "MS.MIL.XPND.GD.ZS": 1,
    "EG.ELC.ACCS.RU.ZS": 0.1,
    "EG.USE.ELEC.KH.PC": 0.9,
    "SH.DYN.MORT":       1.25,
    "SH.MED.BEDS.ZS":    1,
    "GC.TAX.TOTL.GD.ZS": 1,
    "IQ.CPA.FISP.XQ":    0.7
}
CATEGORIES_WEIGHT = {
    "Coercion":       1.3,
    "Infrastructure": 1.4,
    "Taxes":          1.2
}

In [3]:
def min_max_normalization(v, reverse=False):
    """ Apply min-max normalization per year & indicator. """
    
    formula = (v - v.min()) / (v.max() - v.min())
    if reverse:
        return 1 - formula
    else:
        return formula

In [4]:
data = pd.read_excel("./../data/interim/world_bank_data.xlsx")
data.shape

(2304, 7)

In [5]:
data.sample(10)

,iso3,indicator,id,year,value,region,category
1619,MDA,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,8.190000,Eastern Europe,Infrastructure
21,AZE,Armed forces personnel (% of total labor force),MS.MIL.TOTL.TF.ZS,2013,1.742619,Transcaucasia,Coercion
1378,TJK,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,2006,52.400000,Central Asia,Infrastructure
56,EST,Armed forces personnel (% of total labor force),MS.MIL.TOTL.TF.ZS,2014,0.850645,Baltic States,Coercion
1590,LVA,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,2010,5.700000,Baltic States,Infrastructure
2051,AZE,CPIA fiscal policy rating (1=low to 6=high),IQ.CPA.FISP.XQ,1999,NaN,Transcaucasia,Taxes
188,RUS,Armed forces personnel (% of total labor force),MS.MIL.TOTL.TF.ZS,2008,1.941873,Eurasia,Coercion
985,KGZ,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,2003,1644.359844,Central Asia,Infrastructure
1405,UKR,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,2015,9.500000,Eastern Europe,Infrastructure
1925,RUS,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,1999,11.242873,Eurasia,Taxes


In [6]:
# nan policy?
# df = data.loc[data["value"].notnull()].copy()
df = data.copy()
df["proxy_weight"] = df["id"].map(PROXIES_WEIGHT)
df.shape

(2304, 8)

In [7]:
# positive-negative indicators
positives = df.loc[~df["id"].isin(NEGATIVES)].copy()
negatives = df.loc[df["id"].isin(NEGATIVES)].copy()

In [8]:
positives["norm"] = positives.groupby(["year", "indicator"])["value"].transform(min_max_normalization)
negatives["norm"] = negatives.groupby(["year", "indicator"])["value"].transform(min_max_normalization, reverse=True)
stateness = pd.concat([positives, negatives], ignore_index=False).sort_values(["year", "id"])
stateness.loc[stateness["year"].eq(1999) & stateness["id"].eq("SH.MED.BEDS.ZS")]

,iso3,indicator,id,year,value,region,category,proxy_weight,norm
1457,ARM,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,6.20,Transcaucasia,Infrastructure,1.0,0.111724
1475,AZE,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,8.90,Transcaucasia,Infrastructure,1.0,0.484138
1493,BLR,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,12.64,Eastern Europe,Infrastructure,1.0,1.000000
1511,EST,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,7.53,Baltic States,Infrastructure,1.0,0.295172
1529,GEO,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,5.69,Transcaucasia,Infrastructure,1.0,0.041379
1547,KAZ,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,7.25,Central Asia,Infrastructure,1.0,0.256552
1565,KGZ,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,7.47,Central Asia,Infrastructure,1.0,0.286897
1583,LTU,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,9.85,Baltic States,Infrastructure,1.0,0.615172
1601,LVA,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,9.03,Baltic States,Infrastructure,1.0,0.502069
1619,MDA,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,1999,8.19,Eastern Europe,Infrastructure,1.0,0.386207


In [10]:
stateness["proxy_score"] = stateness["norm"] * stateness["proxy_weight"] 
g = stateness.groupby(["year", "iso3", "category"], as_index=False)["proxy_score"].sum()
g["weighted_proxy_score"] = g["proxy_score"] / sum(PROXIES_WEIGHT.values())
g.drop("proxy_score", 1)

,year,iso3,category,weighted_proxy_score
0,1999,ARM,Coercion,0.300236
1,1999,ARM,Infrastructure,0.142876
2,1999,ARM,Taxes,0.000000
3,1999,AZE,Coercion,0.168107
4,1999,AZE,Infrastructure,0.115414
...,...,...,...,...
859,2016,UKR,Infrastructure,0.164100
860,2016,UKR,Taxes,0.099600
861,2016,UZB,Coercion,0.000000
862,2016,UZB,Infrastructure,0.107488


In [12]:
g["category_weight"] = g["category"].map(CATEGORIES_WEIGHT)
g["category_score"] = g["weighted_proxy_score"] * g["category_weight"]
index = g.groupby(["year", "iso3"], as_index=False)["category_score"].sum()
index["stateness"] = index["category_score"] / sum(CATEGORIES_WEIGHT.values()) * 100
result = index.drop("category_score", 1)
result.loc[result["year"].eq(2004)]

,year,iso3,stateness
80,2004,ARM,16.328147
81,2004,AZE,12.035617
82,2004,BLR,23.745336
83,2004,EST,19.209301
84,2004,GEO,7.694125
85,2004,KAZ,13.346531
86,2004,KGZ,6.803466
87,2004,LTU,18.397211
88,2004,LVA,16.757598
89,2004,MDA,11.424670


---

In [13]:
polity = pd.read_excel("./../data/interim/P5_composite_indicators.xlsx")
polity.sample(10)

,year,iso3,democ,autoc,durable,polity,polity2
150,2001,KGZ,1,4,10,-3,-3
140,1991,KGZ,1,4,0,-3,-3
50,2013,AZE,0,7,18,-7,-7
203,1998,LVA,8,0,7,8,8
29,1992,AZE,3,2,0,1,1
224,1991,LTU,10,0,0,10,10
111,2018,EST,9,0,18,9,9
307,2006,SRB,9,1,0,8,8
406,1993,UZB,0,9,2,-9,-9
316,2015,SRB,9,1,9,8,8


In [14]:
polity["polity2_norm"] = polity.groupby("year")["polity2"].transform(min_max_normalization)
polity["polity2_norm"] *= 100

In [15]:
m = pd.merge(
    result, 
    polity[["iso3", "year", "polity2_norm"]],
    how="left",
    on=["year", "iso3"]
)
m.sample(10)

,year,iso3,stateness,polity2_norm
151,2008,LTU,15.616459,100.000000
286,2016,UKR,13.605215,68.421053
39,2001,LTU,16.335424,100.000000
221,2012,TKM,4.771216,0.000000
218,2012,RUS,20.120961,68.421053
165,2009,KAZ,11.767332,15.789474
90,2004,RUS,23.407543,78.947368
262,2015,KGZ,8.369930,84.210526
197,2011,KAZ,14.068751,15.789474
246,2014,KGZ,8.799340,84.210526


In [16]:
m.to_excel(f"./../data/processed/stateness_{DATE}.xlsx", index=False)